In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from IPython.display import display
from sklearn.preprocessing import StandardScaler
#force pandas to show all df cols instead of (...)
pd.options.display.max_columns = None

In [2]:
#bring in raw data
df = pd.read_csv('./train1percentsample.txt', delimiter='\t', header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0,1.0,3,1.0,1.0,1369.0,2.0,2.0,3.0,2.0,0.0,1.0,0.0,1.0,68fd1e64,c2004197,5a2cae95,28d926b8,a93acb09,fe6b92e5,bb3f8e2d,0b153874,a73ee510,fa7d0797,cf0467b4,ffc278cb,9ded12ab,07d13a8f,3af33a20,fc53f85c,27c07bd6,c6c8223f,NaN,NaN,ed35ed93,NaN,3a171ecb,4fcc135f,NaN,NaN
1,0,NaN,37,14.0,14.0,6979.0,22.0,1.0,23.0,22.0,NaN,1.0,NaN,14.0,05db9164,04440d29,68c14318,bc60eb26,4cf72387,NaN,675e81f6,5b392875,a73ee510,29cfc193,4a77ddca,cfa59fde,dc1d72e4,1adce6ef,d0ab1707,853c37ce,07c540c4,15f0df20,NaN,NaN,c0138135,NaN,32c7478e,bb9a5ea6,NaN,NaN
2,0,7.0,1,NaN,0.0,46.0,26.0,7.0,1.0,0.0,1.0,1.0,NaN,0.0,68fd1e64,f0cf0024,d406ed6f,ddb90d33,4cf72387,fbad5c96,b226f465,5b392875,a73ee510,0506bfd4,c6c91669,28b79840,b8a76289,b28479f6,fdb1071f,5ea15ec1,e5ba7672,cc693e93,21ddcdc9,b1252a9d,0366ddd4,NaN,32c7478e,90b99a4b,ea9a246c,ab7f43ef
3,0,9.0,1,3.0,14.0,5.0,7.0,66.0,18.0,77.0,1.0,4.0,NaN,7.0,5bfa8ab5,e3db0bac,02ac7cbb,bbb6c1e5,25c83c98,7e0ccccf,93ec533b,0b153874,a73ee510,3b08e48b,9ffb3655,fe390287,a0874a81,07d13a8f,618b0ee5,877e1719,e5ba7672,a7ccaded,21ddcdc9,5840adea,b3f82dff,NaN,dbb486d7,8062ea81,2bf691b1,b29c74dc
4,1,3.0,837,1.0,NaN,1559.0,1.0,25.0,1.0,392.0,0.0,7.0,0.0,NaN,ae82ea21,1cfdf714,b7c4dea5,13508380,25c83c98,fbad5c96,7195046d,0b153874,a73ee510,d5972498,4d8549da,0ee8e9b4,51b97b8f,07d13a8f,e439dd9b,9d1967d0,3486227d,9df49ecd,55dd3565,b1252a9d,f80bb5e3,NaN,3a171ecb,45ab94c8,cb079c2d,c84c4aec


In [4]:
#check shape
df.shape

(458406, 40)

In [5]:
#give cols name for easier access
cols = ['col_'+str(i) for i in range(40)]
df.columns = cols

#check class balance; imbalanced; rebalance after eliminating nulls
df['col_0'].value_counts()

0    340662
1    117744
Name: col_0, dtype: int64

In [6]:
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39
0,0,1.0,3,1.0,1.0,1369.0,2.0,2.0,3.0,2.0,0.0,1.0,0.0,1.0,68fd1e64,c2004197,5a2cae95,28d926b8,a93acb09,fe6b92e5,bb3f8e2d,0b153874,a73ee510,fa7d0797,cf0467b4,ffc278cb,9ded12ab,07d13a8f,3af33a20,fc53f85c,27c07bd6,c6c8223f,NaN,NaN,ed35ed93,NaN,3a171ecb,4fcc135f,NaN,NaN
1,0,NaN,37,14.0,14.0,6979.0,22.0,1.0,23.0,22.0,NaN,1.0,NaN,14.0,05db9164,04440d29,68c14318,bc60eb26,4cf72387,NaN,675e81f6,5b392875,a73ee510,29cfc193,4a77ddca,cfa59fde,dc1d72e4,1adce6ef,d0ab1707,853c37ce,07c540c4,15f0df20,NaN,NaN,c0138135,NaN,32c7478e,bb9a5ea6,NaN,NaN
2,0,7.0,1,NaN,0.0,46.0,26.0,7.0,1.0,0.0,1.0,1.0,NaN,0.0,68fd1e64,f0cf0024,d406ed6f,ddb90d33,4cf72387,fbad5c96,b226f465,5b392875,a73ee510,0506bfd4,c6c91669,28b79840,b8a76289,b28479f6,fdb1071f,5ea15ec1,e5ba7672,cc693e93,21ddcdc9,b1252a9d,0366ddd4,NaN,32c7478e,90b99a4b,ea9a246c,ab7f43ef
3,0,9.0,1,3.0,14.0,5.0,7.0,66.0,18.0,77.0,1.0,4.0,NaN,7.0,5bfa8ab5,e3db0bac,02ac7cbb,bbb6c1e5,25c83c98,7e0ccccf,93ec533b,0b153874,a73ee510,3b08e48b,9ffb3655,fe390287,a0874a81,07d13a8f,618b0ee5,877e1719,e5ba7672,a7ccaded,21ddcdc9,5840adea,b3f82dff,NaN,dbb486d7,8062ea81,2bf691b1,b29c74dc
4,1,3.0,837,1.0,NaN,1559.0,1.0,25.0,1.0,392.0,0.0,7.0,0.0,NaN,ae82ea21,1cfdf714,b7c4dea5,13508380,25c83c98,fbad5c96,7195046d,0b153874,a73ee510,d5972498,4d8549da,0ee8e9b4,51b97b8f,07d13a8f,e439dd9b,9d1967d0,3486227d,9df49ecd,55dd3565,b1252a9d,f80bb5e3,NaN,3a171ecb,45ab94c8,cb079c2d,c84c4aec


In [7]:
#convert categorical features from hex to int
cat_cols = ['col_'+str(i) for i in range(14,40)]
for i in cat_cols:
    #have to only convert non-NaN values, so use notnull
    df[i][pd.notnull(df[i])] = df[i][pd.notnull(df[i])].apply(lambda x: int(x,16))

/Users/benthompson/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
df.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,col_37,col_38,col_39
0,0,1.0,3,1.0,1.0,1369.0,2.0,2.0,3.0,2.0,0.0,1.0,0.0,1.0,1761418852,3254796695,1.51288e+09,6.8532e+08,2839202569,4.26846e+09,3141504557,185940084,2805916944,4202497943,3473172404,4.29093e+09,2649559723,131152527,989018656,4.23336e+09,666926038,3335004735,NaN,NaN,3.97973e+09,NaN,974593739,1.33877e+09,NaN,NaN
1,0,NaN,37,14.0,14.0,6979.0,22.0,1.0,23.0,22.0,NaN,1.0,NaN,14.0,98275684,71568681,1.7575e+09,3.16047e+09,1291264903,NaN,1734246902,1530472565,2805916944,701481363,1249369546,3.48374e+09,3692917476,450684655,3500873479,2.23532e+09,130367684,368107296,NaN,NaN,3.2225e+09,NaN,851920782,3.14746e+09,NaN,NaN
2,0,7.0,1,NaN,0.0,46.0,26.0,7.0,1.0,0.0,1.0,1.0,NaN,0.0,1761418852,4040097828,3.55722e+09,3.71989e+09,1291264903,4.22244e+09,2988897381,1530472565,2805916944,84328404,3335067241,6.83121e+08,3097977481,2995026422,4256237343,1.58763e+09,3854202482,3429449363,5.68184e+08,2.972e+09,5.70731e+07,NaN,851920782,2.42808e+09,3.93597e+09,2.87724e+09
3,0,9.0,1,3.0,14.0,5.0,7.0,66.0,18.0,77.0,1.0,4.0,NaN,7.0,1543146165,3822783404,4.48586e+07,3.14932e+09,633879704,2.11477e+09,2481738555,185940084,2805916944,990438539,2684040789,4.26515e+09,2693220993,131152527,1636503269,2.27319e+09,3854202482,2815208941,5.68184e+08,1.48063e+09,3.01939e+09,NaN,3686041303,2.15397e+09,7.37579e+08,2.9966e+09
4,1,3.0,837,1.0,NaN,1559.0,1.0,25.0,1.0,392.0,0.0,7.0,0.0,NaN,2927815201,486405908,3.08313e+09,3.24044e+08,633879704,4.22244e+09,1905591405,185940084,2805916944,3583452312,1300580826,2.50145e+08,1371110287,131152527,3828997531,2.63569e+09,881205885,2650054349,1.44056e+09,2.972e+09,4.16152e+09,NaN,974593739,1.16887e+09,3.40627e+09,3.36044e+09


In [9]:
#get list of column names
df.columns

Index(['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7',
       'col_8', 'col_9', 'col_10', 'col_11', 'col_12', 'col_13', 'col_14',
       'col_15', 'col_16', 'col_17', 'col_18', 'col_19', 'col_20', 'col_21',
       'col_22', 'col_23', 'col_24', 'col_25', 'col_26', 'col_27', 'col_28',
       'col_29', 'col_30', 'col_31', 'col_32', 'col_33', 'col_34', 'col_35',
       'col_36', 'col_37', 'col_38', 'col_39'],
      dtype='object')

In [10]:
#remove cols that are mainly null - decision based on Matt's EDA in DataCleaningTest-2
continuous = ['col_0', 'col_2', 'col_5', 'col_7',
              'col_8', 'col_9','col_11']
categorical= ['col_14','col_15', 'col_18', 'col_19',
              'col_20', 'col_21','col_22', 'col_24',
              'col_26', 'col_27', 'col_28','col_30',
              'col_31','col_36']

#make df of continuous vars to impute mean
df_continuous = df[continuous].copy()
#impute mean
df_continuous.fillna(df_continuous.mean(), inplace=True)
#make df of categorical vars
df_categorical = df[categorical].copy()
#merge dfs
df_combined = df_continuous.merge(df_categorical, left_index=True, right_index=True)
#drop remaining nans (from cat features)
df_combined.dropna(inplace=True)

In [11]:
#make sure there are no nans
df_combined.describe()

,col_0,col_2,col_5,col_7,col_8,col_9,col_11
count,403172.000000,403172.000000,4.031720e+05,403172.000000,403172.000000,403172.000000,403172.000000
mean,0.256546,107.800286,1.895491e+04,16.807340,12.726923,110.105448,2.775203
std,0.436727,393.608851,7.084429e+04,67.059618,16.727682,222.266438,5.154588
min,0.000000,-2.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,3.520000e+02,1.000000,2.000000,12.000000,1.000000
50%,0.000000,3.000000,2.888000e+03,4.000000,7.000000,44.000000,1.000000
75%,1.000000,37.000000,1.149900e+04,14.000000,19.000000,108.000000,3.000000
max,1.000000,19503.000000,2.633976e+06,12309.000000,3444.000000,13879.000000,144.000000


In [16]:
#limit cardinality of most cat columns to 2 (most common value and not)
for col in categorical:
    #Keep values for any col w/<30 distinct values otherwise
    #reduce to most frequenct value as 1 and others as 0
    if len(getattr(df_combined,col).unique()) > 30:
        #find most common value
        top_value = df_combined[col].value_counts().index[0]
        #set most common value to 1 and all others to 0
        one_hot = np.where(df_combined[col]==top_value,1,0)
        #set new values to old column name
        df_combined[col] = one_hot

In [17]:
#one hot encode all cat columns
dummies = pd.get_dummies(df_combined.iloc[:,7:], prefix=categorical, columns=categorical)

In [18]:
dummies.head()

,col_14_0,col_14_1,col_15_0,col_15_1,col_18_0,col_18_1,col_19_326208445.0,col_19_1006043623.0,col_19_1011281330.0,col_19_1869454312.0,col_19_2114768079.0,col_19_3226958037.0,col_19_3345673462.0,col_19_3813803042.0,col_19_4015688902.0,col_19_4059225645.0,col_19_4222442646.0,col_19_4260457412.0,col_19_4268462821.0,col_20_0,col_20_1,col_21_0,col_21_1,col_22_2093428418,col_22_2709664746,col_22_2805916944,col_24_0,col_24_1,col_26_0,col_26_1,col_27_85072358,col_27_100783029,col_27_131152527,col_27_197642434,col_27_450684655,col_27_607800936,col_27_648838388,col_27_847330849,col_27_1525414787,col_27_1690912869,col_27_1750341970,col_27_1753086708,col_27_2364459976,col_27_2435002992,col_27_2510728076,col_27_2876477673,col_27_2904344950,col_27_2995026422,col_27_3474965645,col_27_3488554025,col_27_3537889393,col_27_3705103086,col_27_3906789498,col_27_3961124128,col_27_4156666687,col_27_4167234145,col_28_0,col_28_1,col_30_130367684,col_30_512280399,col_30_537242577,col_30_666926038,col_30_881205885,col_30_2003624857,col_30_2399067775,col_30_2942138380,col_30_3569056728,col_30_3854202482,col_31_0,col_31_1,col_36_635684715,col_36_851920782,col_36_974593739,col_36_1111468905,col_36_1440560485,col_36_1918445973,col_36_2245372309,col_36_2478494400,col_36_2992939104,col_36_3168725356,col_36_3195814324,col_36_3286002927,col_36_3353110304,col_36_3686041303
0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
#combine dummies and continuous features
final_features = df_combined.iloc[:,:7].merge(dummies, left_index=True, right_index=True)

In [20]:
final_features.head()

,col_0,col_2,col_5,col_7,col_8,col_9,col_11,col_14_0,col_14_1,col_15_0,col_15_1,col_18_0,col_18_1,col_19_326208445.0,col_19_1006043623.0,col_19_1011281330.0,col_19_1869454312.0,col_19_2114768079.0,col_19_3226958037.0,col_19_3345673462.0,col_19_3813803042.0,col_19_4015688902.0,col_19_4059225645.0,col_19_4222442646.0,col_19_4260457412.0,col_19_4268462821.0,col_20_0,col_20_1,col_21_0,col_21_1,col_22_2093428418,col_22_2709664746,col_22_2805916944,col_24_0,col_24_1,col_26_0,col_26_1,col_27_85072358,col_27_100783029,col_27_131152527,col_27_197642434,col_27_450684655,col_27_607800936,col_27_648838388,col_27_847330849,col_27_1525414787,col_27_1690912869,col_27_1750341970,col_27_1753086708,col_27_2364459976,col_27_2435002992,col_27_2510728076,col_27_2876477673,col_27_2904344950,col_27_2995026422,col_27_3474965645,col_27_3488554025,col_27_3537889393,col_27_3705103086,col_27_3906789498,col_27_3961124128,col_27_4156666687,col_27_4167234145,col_28_0,col_28_1,col_30_130367684,col_30_512280399,col_30_537242577,col_30_666926038,col_30_881205885,col_30_2003624857,col_30_2399067775,col_30_2942138380,col_30_3569056728,col_30_3854202482,col_31_0,col_31_1,col_36_635684715,col_36_851920782,col_36_974593739,col_36_1111468905,col_36_1440560485,col_36_1918445973,col_36_2245372309,col_36_2478494400,col_36_2992939104,col_36_3168725356,col_36_3195814324,col_36_3286002927,col_36_3353110304,col_36_3686041303
0,0,3,1369.0,2.0,3.0,2.0,1.0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,46.0,7.0,1.0,0.0,1.0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,5.0,66.0,18.0,77.0,4.0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,837,1559.0,25.0,1.0,392.0,7.0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,-1,7463.0,1.0,0.0,0.0,1.0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [21]:
#check class balance
final_features['col_0'].value_counts()

0    299740
1    103432
Name: col_0, dtype: int64

In [22]:
#reduce 0 class
df_0 = final_features[final_features['col_0']==0]
#sample
df_0_sampled = df_0.sample(n=103432)
#check size
df_0_sampled.shape

(103432, 91)

In [23]:
df_0_sampled['col_0'].value_counts()

0    103432
Name: col_0, dtype: int64

In [24]:
#create df of balanced groups
final_features = final_features[final_features['col_0']==1]
#add back in 0 class
final_features=pd.concat([final_features,df_0_sampled])
#shuffle
final_features.sample(frac=1)
#check final_features
final_features.shape

(206864, 91)

In [25]:
#check class balance
final_features['col_0'].value_counts()

1    103432
0    103432
Name: col_0, dtype: int64

In [26]:
#convert to numpy
X = final_features.iloc[:,1:].values
y = final_features.iloc[:,:1].values
#reshape y
y = y.ravel()
#scale data
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [27]:
#L1 for important features
l1_clf = LogisticRegression(penalty='l1')
l1_clf.fit(X, y)

/Users/benthompson/anaconda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
#get coefficients for all features
coef_sq = l1_clf.coef_

#get index of top N values for each class
weight_indx = np.argsort(coef_sq)[:, -91:]

#flatten so can use to look up wieghts
weight_indx = weight_indx.flatten()

#get coefficients based on index
weights = coef_sq[:, weight_indx]

#get features that match weights based on index
#need to offset by 1 bc label was col_0
features = final_features.columns[1:][weight_indx]

# # make table
df = pd.DataFrame({'weights': weights[0]}
                  , index=features)
df.iloc[::-1]

,weights
col_11,0.478119
col_36_1111468905,0.143814
col_30_3854202482,0.136948
col_27_1690912869,0.127291
col_30_2399067775,0.085073
col_2,0.082544
col_27_85072358,0.076250
col_27_847330849,0.071116
col_19_1869454312.0,0.067690
col_36_851920782,0.064650


In [29]:
#CV L2 for accuracy refernce
l2_clf = LogisticRegression()
scores = cross_val_score(l2_clf, X, y, cv=5, n_jobs=-1)
scores

array([ 0.63856528,  0.63970126,  0.64106159,  0.63978053,  0.63487383])